In [1]:
import cv2

In [2]:
print("face" in dir(cv2))  # Should return True if contrib is installed


True


In [3]:
import numpy as np

In [4]:
import os

In [5]:
from PIL import Image

In [6]:
pip install opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --user --upgrade opencv-contrib-python


Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import os

def create_user(face_id, name):
    web = cv2.VideoCapture(0)
    web.set(3, 640)
    web.set(4, 480)

    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    f_dir = 'dataset'
    path = os.path.join(f_dir, name)
    if not os.path.isdir(path):
        os.makedirs(path)

    counter = 0
    while True:
        ret, img = web.read()
        if not ret:
            break

        img = cv2.flip(img, 1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        multi_face = faces.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in multi_face:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            counter += 1

            face_img = gray[y:y+h, x:x+w]
            filename = f"{name}_{face_id}_{counter}.jpg"
            cv2.imwrite(os.path.join(path, filename), face_img)

            cv2.imshow("Image", img)

        k = cv2.waitKey(100) & 0xff
        if k == 27 or counter >= 40:
            break

    web.release()
    cv2.destroyAllWindows()


In [9]:
create_user(1,"Gauri")

In [10]:
create_user(2,"Vasu")

In [11]:

def train():
    database = 'dataset'
    img_dir=[x[0] for x in os.walk(database)][1::]

    recognizer = cv2.face.LBPHFaceRecognizer_create()
    detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    faceSamples = []
    ids = []

    img_dirs = [os.path.join(database, folder) for folder in os.listdir(database)]

    for path in img_dirs:
        imagePaths = [os.path.join(path, f) for f in os.listdir(path)]

        for imagePath in imagePaths:
            PIL_img = Image.open(imagePath).convert('L')  # Convert to grayscale
            img_numpy = np.array(PIL_img, 'uint8')

            try:
                # Extract ID from filename: name_id_counter.jpg
                filename = os.path.basename(imagePath)
                id = int(filename.split('_')[1])
            except:
                print(f"[WARN] Skipping file due to bad format: {filename}")
                continue

            faces = detector.detectMultiScale(img_numpy)

            for (x, y, w, h) in faces:
                faceSamples.append(img_numpy[y:y+h, x:x+w])
                ids.append(id)

    if len(faceSamples) == 0:
        print("[ERROR] No faces found to train!")
        return 0

    recognizer.train(faceSamples, np.array(ids))
    recognizer.write('trainer.yml')

    print(f'\n[INFO] {len(np.unique(ids))} unique face(s) trained. Exiting program.')
    return len(np.unique(ids))

In [12]:
train()


[INFO] 2 unique face(s) trained. Exiting program.


2

In [13]:
import cv2

def recognizer(names):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer.yml')

    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    font = cv2.FONT_HERSHEY_SIMPLEX
    face_count = 0

    cam = cv2.VideoCapture(0)
    cam.set(3, 640)
    cam.set(4, 480)

    minW = 0.1 * cam.get(3)
    minH = 0.1 * cam.get(4)

    while True:
        ret, img = cam.read()
        if not ret:
            break

        img = cv2.flip(img, 1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(int(minW), int(minH))
        )

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            id_, confidence = recognizer.predict(gray[y:y+h, x:x+w])

            if confidence < 100:
                name = names.get(id_, "Unknown")
                confidence_text = f"  {round(100 - confidence)}%"
            else:
                name = "Unknown"
                confidence_text = f"  {round(100 - confidence)}%"

            cv2.putText(img, str(name), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, str(confidence_text), (x + 5, y + h + 20), font, 1, (255, 255, 0), 1)

        cv2.imshow('Camera', img)

        k = cv2.waitKey(10) & 0xff
        if k == 27:
            break

    cam.release()
    cv2.destroyAllWindows()


In [ ]:
recognizer({1:"Gauri"})

In [ ]:
recognizer({2:"Vasu"})